# Using distorter

The concept of cation mutation (ion substitution) to tune the properties of compounds has been around at least since the [1950s](https://www.sciencedirect.com/science/article/pii/0022369758900507) and has proved a useful concept in more [more recent studies](https://journals.aps.org/prb/abstract/10.1103/PhysRevB.79.165211). 

In this notebook, we demonstrate the process of generating all possible (symmetry inequivalent) substitutions of Sr on Ba sites in the cubic perovskite BaTiO<sub>3</sub>, considering both single and double substitutions. The distorter module leverages the [ASE](https://wiki.fysik.dtu.dk/ase) Python library.

In [3]:
import smact.builder as builder
import smact.distorter as distorter
import numpy as np

# Step 1: Build the initial cubic perovskite structure
smact_lattice, test_case = builder.cubic_perovskite(
    ["Ba", "Ti", "O"], repetitions=[2, 2, 2]
)

# Step 2: Perform a single substitution
subs_site = [0.0, 0.0, 0.0]
single_substitution = distorter.make_substitution(test_case, subs_site, "Sr")

# Step 3: Build a sub-lattice for disorder
sub_lattice = distorter.build_sub_lattice(single_substitution, "Ba")

# Step 4: Extract the required data
lattice_matrix = single_substitution.get_cell()  # Get the lattice matrix
positions = (
    single_substitution.get_scaled_positions()
)  # Get scaled positions (fractional coordinates)
atomic_numbers = single_substitution.get_atomic_numbers()  # Get atomic numbers

# Package the data into the expected format: (lattice, positions, atomic numbers)
lattice_data = (lattice_matrix, positions, atomic_numbers)

# Step 5: Enumerate inequivalent sites using the properly formatted data
inequivalent_sites = distorter.get_inequivalent_sites(sub_lattice, lattice_data)

# Step 6: Replace Ba at inequivalent sites with Sr and store the structures
substituted_structures = []

for i, inequivalent_site in enumerate(inequivalent_sites):
    print(
        f"Substitution {i + 1}: Replacing Ba with Sr at site {inequivalent_site}"
    )
    distorted = distorter.make_substitution(
        single_substitution, inequivalent_site, "Sr"
    )
    substituted_structures.append(
        distorted
    )  # Store the structure for visualization
    print("-" * 40)

print("Substitutions completed")

Substitution 1: Replacing Ba with Sr at site [0.  0.  0.5]
----------------------------------------
Substitution 2: Replacing Ba with Sr at site [0.  0.5 0.5]
----------------------------------------
Substitution 3: Replacing Ba with Sr at site [0.5 0.  0. ]
----------------------------------------
Substitution 4: Replacing Ba with Sr at site [0.5 0.  0.5]
----------------------------------------
Substitution 5: Replacing Ba with Sr at site [0.5 0.5 0.5]
----------------------------------------
Substitutions completed


In [ ]:
# The distorted objects are of the ase atoms class,
# so can be written out to a crystal structure file like a cif
distorted.write("example_output.cif")